This code is a motion detection algorithm that alarms you when you have an unwanted visitor in your home.

In [3]:
import argparse
import datetime
import imutils
import time
import cv2
                                                            # this is for the reference frame that we compare everything to
video_capture = cv2.VideoCapture(0)                         # turn camera on
ret, frame = video_capture.read()                           # get frame
background = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)        # grayscale frame
background = cv2.GaussianBlur(background, (5,5), 0)         # blur the image to deal with noise

width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))    # get resolution of camera
height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
filename = "surveillance.avi"                               # name of video recording
fps = 24                                                    # FPS of video recording
fourcc = cv2.VideoWriter_fourcc(*"XVID")                    # video encoding
out=cv2.VideoWriter(filename, fourcc, fps, (width, height)) # output video

while True:
    ret, frame = video_capture.read()              # get frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # grayscale
    gray = cv2.GaussianBlur(gray, (5,5), 0)        # blur the image to deal with noise
    
    frameDelta = cv2.absdiff(background, gray)     # delta = abs(background frame - current frame)
    
    thresh = cv2.threshold(frameDelta, 15, 255, cv2.THRESH_BINARY)[1] # if the delta is greater than 25 make it white else make it black
    thresh = cv2.dilate(thresh, None, iterations=2)
    
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # for finding countours of intruder
    cnts = imutils.grab_contours(cnts)
    
    for c in cnts:
    # if the contour is too small, ignore it
        if cv2.contourArea(c) < 3000:
            text = "SAFE"
            print("No motion detected (0)")    # if there are no major changes, there are no contours, there's no intruder
        else:
            (x, y, w, h) = cv2.boundingRect(c) # get and show a bounding box of intruder
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            text = "UNSAFE"
            print("Motion detected (1)")
    
    # draw the text and timestamp on the frame
    cv2.putText(frame, "Room Status: {}".format(text), (frame.shape[1]-175, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"), (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)
    
    # Record Vid
    
    # ret (bool return value on webcam read function), so if we can read the webcam properly
    if ret == True:
        out.write(background)
    
    cv2.imshow("Security Feed", frame)
    cv2.imshow("Thresh", thresh)
    cv2.imshow("Frame Delta", frameDelta)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()
video_capture.release()

NameError: name 'text' is not defined